---
# Set 1

##### Basics
---

### Challenge 1

Convert hex to base64

In [1]:
from base64 import b64encode

def hex_to_base64(hexstring):
    byte_seq = bytes.fromhex(hexstring)
    return b64encode(byte_seq)

hex_to_base64('49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d').hex()

'535364744947747062477870626d63676557393163694269636d4670626942736157746c4947456763473970633239756233567a494731316332687962323974'

### Challenge 2

**Fixed XOR:**
Write a function that takes two equal-length buffers and produces their XOR combination.

In [2]:
def fixed_xor(byte_seq1, byte_seq2):
    xor = b''
    for i, j in zip(byte_seq1, byte_seq2):
        xor += bytes([i ^ j])
    return xor

def main():
    byte_seq1 = bytes.fromhex('1c0111001f010100061a024b53535009181c')
    byte_seq2 = bytes.fromhex('686974207468652062756c6c277320657965')
    print(fixed_xor(byte_seq1, byte_seq2))
    
main()

b"the kid don't play"


### Challenge 3

Single-byte XOR cipher

In [3]:
def get_score(input_bytes):
    letter_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    return sum([letter_frequencies.get(chr(byte), 0) for byte in input_bytes.lower()])

def break_sbyte_xor(byte_seq):
    potential_messages = []
    
    for key_value in range(256):  # each ascii character
        message = b''
        
        for byte in byte_seq:
            xor = bytes([key_value ^ byte])
            message += xor
        
        data = {'Message': message, 'Key': key_value, 'Score': get_score(message)}
        potential_messages.append(data)
        
    best_score = sorted(potential_messages, key=lambda x: x['Score'], reverse=True)[0]
    return best_score

break_sbyte_xor(bytes.fromhex('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'))

{'Message': b"Cooking MC's like a pound of bacon", 'Key': 88, 'Score': 2.14329}

### Challenge 4

Detect single-character XOR

In [4]:
from urllib.request import urlopen

def detect_sbyte_xor(lines):
    potential_plaintext = []
    
    for line in lines:
        # Use break_sbyte_xor() method from Challenge 3
        potential_plaintext.append(break_sbyte_xor(line))
    
    best_line = sorted(potential_plaintext, key=lambda x: x['Score'], reverse=True)[0]
    return best_line

def main():
    lines = []
    with urlopen('https://cryptopals.com/static/challenge-data/4.txt') as data:
        for line in data:
            lines.append(bytes.fromhex(line.rstrip().decode()))
    print(detect_sbyte_xor(lines))

main()

{'Message': b'Now that the party is jumping\n', 'Key': 53, 'Score': 2.03479}


### Challenge 5

Implement repeating-key XOR (Vignere Cipher)

In [5]:
def repeating_key_xor(message, key):
    message = message
    key = key
    ciphertext = b''
    
    for i in range(len(message)):
        m = message[i]  # current byte from message
        k = key[i % len(key)]  # byte of key for XOR
        
        xor = bytes([m ^ k])
        ciphertext += xor
    
    return ciphertext

repeating_key_xor(b"Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal", b"ICE").hex()

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20690a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

### Challenge 6

Break repeating-key XOR (Vigenere Cipher)

In [6]:
from urllib.request import urlopen
import numpy as np
from itertools import combinations
from base64 import b64decode

def hamming(block1, block2):
    """Returns the Hamming distance between two equal length byte strings"""
    distance = 0
    for byte1, byte2 in zip(block1, block2):
        byte1 = bin(byte1)[2:].zfill(8).encode()
        byte2 = bin(byte2)[2:].zfill(8).encode()
        distance += sum([1 for bit1, bit2 in zip(byte1, byte2) if bit1 != bit2])
    return distance

def break_repeating_key_xor(ciphertext):
    # Find the keysize, taking advantage of the fact that the Hamming distance between
    # Engligh characters will, on average, be less than the distance between random bytes.
    avg_norm_hd = []  # Averaged normilized Hamming distance
    for keysize in range(2, 41):
        blocks = [ciphertext[i:i+keysize] for i in range(0, len(ciphertext), keysize)]  # break message into keysize-sized blocks
        norm_hd = []  # Normalized Hamming distance
        
        for block1, block2 in combinations(blocks, 2):  # compute the Hamming distance between each block
            norm_hd.append(hamming(block1, block2) / keysize)
        avg_norm_hd.append({'Keysize': keysize, 'Score': np.mean(norm_hd)})
        
    avg_norm_hd = sorted(avg_norm_hd, key=lambda x: x['Score'])[:3]  # Top three results for the keysize
    
    # For each of the top 3 keysizes, transpose the blocks (i.e. chunk ciphertext into blocks with multiples of keysize, then keysize + 1, etc...).
    # Then solve each block with single byte xor, and combine the resulting keys to form the potential key for that keysize.
    # The score for the final key will be the score for all the single byte keys added together. Rank the final keys, and decrypt using the best one.
    possible_key_list = []
    for item in avg_norm_hd:
        keysize = item['Keysize']
        possible_key = b''
        key_score = 0
        
        # Transpose the blocks
        for i in range(keysize):
            block = b''
            for j in range(i, len(ciphertext), keysize):
                block += bytes([ciphertext[j]])
            
            key_info = break_sbyte_xor(block)  # Use break_sbyte_xor() method from Challenge 3
            possible_key += chr(key_info['Key']).encode()  # add single byte key to potential key
            key_score += key_info['Score']  # add score for single byte key to the score for the potential key
        
        possible_key_list.append({'Key': possible_key, 'Score': key_score})
    
    # Sort potential keys, and decrypt using the best one
    best_key = sorted(possible_key_list, key=lambda x: x['Score'], reverse=True)[0]['Key']
    message = repeating_key_xor(ciphertext, best_key)  # Use repeating_key_xor() method from Challenge 5
    return {'Message': message, 'Key': best_key}

def main():
    # Load message, removing new line characters along the way
    ciphertext = b''
    with urlopen('https://cryptopals.com/static/challenge-data/6.txt') as data:
        for line in data:
            ciphertext += b64decode(line.rstrip())
    print(break_repeating_key_xor(ciphertext))

main()

{'Message': b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \

### Challenge 7

AES in ECB mode

In [7]:
from urllib.request import urlopen
from base64 import b64decode
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def AES_ECB_encrypt(plaintext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return ciphertext

def AES_ECB_decrypt(ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

def main():
    ciphertext = b''
    with urlopen('https://cryptopals.com/static/challenge-data/7.txt') as data:
        for line in data:
            ciphertext += b64decode(line.rstrip())
    print(AES_ECB_decrypt(ciphertext, b'YELLOW SUBMARINE'))
    
    # The following is a test of the encrypt and decrypt for Challenge 10.
    # It uses the padding method from Challenge 9
#     message = PKCS7_pad(
#         b'It was the best of times, it was the worst of times, it was the age of wisdom, '
#         b'it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, '
#         b'it was the season of Light, it was the season of Darkness, it was the spring of hope, '
#         b'it was the winter of despair.', 16
#     )
#     key = b'YELLOW SUBMARINE'
#     ciphertext = AES_ECB_encrypt(message, key)
#     plaintext = PKCS7_unpad(AES_ECB_decrypt(ciphertext, key))
#     print(plaintext)

main()

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

### Challenge 8

Detect AES in ECB mode

In [8]:
from urllib.request import urlopen

def detect_AES_ECB(ciphertext, keysize=16):
    blocks = [ciphertext[i:i+keysize] for i in range(0, len(ciphertext), keysize)]
    return len(blocks) - len(set(blocks))

def main():
    scores = []
    ciphertext_list = []
    with urlopen('https://cryptopals.com/static/challenge-data/8.txt') as data:
        for line in data:
            ciphertext = bytes.fromhex(line.rstrip().decode())
            scores.append(detect_AES_ECB(ciphertext))
            ciphertext_list.append(ciphertext)
    
    best_score = max(scores)
    print(f"ECB mode detected on line {scores.index(best_score) + 1}\nNumber of repetitions = {best_score}")
    print()
    print(ciphertext_list[scores.index(best_score)].hex())
    
main()

ECB mode detected on line 133
Number of repetitions = 3

d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a


---
# Set 2

##### Block Crypto
---

### Challenge 9

Implement PKCS#7 padding

In [9]:
def PKCS7_pad(plaintext, blocksize):
    pad_size = blocksize - (len(plaintext) % blocksize)
    padding = b''.join([bytes([pad_size]) for _ in range(pad_size)])
    return plaintext + padding

def PKCS7_unpad(padded_data):
    pad_size = padded_data[-1]
    unpadded_data = padded_data[:-pad_size]
    return unpadded_data

def main():
    data = b'YELLOW SUBMARINE'
    padded_data = PKCS7_pad(data, 20)
    unpadded_data = PKCS7_unpad(padded_data)
    print(f'{data} -> {padded_data} -> {unpadded_data}')

main()

b'YELLOW SUBMARINE' -> b'YELLOW SUBMARINE\x04\x04\x04\x04' -> b'YELLOW SUBMARINE'


### Challenge 10

Implement CBC mode

In [10]:
import os
from urllib.request import urlopen

def AES_CBC_encrypt(plaintext, key, iv):
    # Key must be 128, 192, or 256 bits, and plaintext must already be padded
    assert len(key) == 16 or len(key) == 24 or len(key) == 32
    assert len(plaintext) % len(key) == 0
    
    blocks = [plaintext[i:i+len(key)] for i in range(0, len(plaintext), len(key))]
    ciphertext_blocks = []
    
    for block in blocks:
        if len(ciphertext_blocks) == 0:
            xor_block = fixed_xor(block, iv)  # fixed_xor() from Challenge 2
        else:
            xor_block = fixed_xor(block, ciphertext_blocks[-1])  # fixed_xor() from Challenge 2
        
        ciphertext_blocks.append(AES_ECB_encrypt(xor_block, key))  # ECB encryption from Challenge 7
    
    ciphertext = b''.join(ciphertext_blocks)
    return ciphertext

def AES_CBC_decrypt(ciphertext, key, iv):
    # Key must be 128, 192, or 256 bits, and ciphertext must already be padded
    assert len(key) == 16 or len(key) == 24 or len(key) == 32
    assert len(ciphertext) % len(key) == 0
    
    blocks = [ciphertext[i:i+len(key)] for i in range(0, len(ciphertext), len(key))]
    plaintext_blocks = []
    
    for i in range(len(blocks)):
        block = blocks[i]
        
        decrypted_block = AES_ECB_decrypt(block, key)  # ECB decryption from Challenge 7
        
        if len(plaintext_blocks) == 0:
            plaintext_blocks.append(fixed_xor(decrypted_block, iv))  # fixed_xor() from Challenge 2
        else:
            plaintext_blocks.append(fixed_xor(decrypted_block, blocks[i - 1]))  # fixed_xor() from Challenge 2
    
    plaintext = b''.join(plaintext_blocks)
    return plaintext
            
def main():
    key = b'YELLOW SUBMARINE'
    iv = os.urandom(len(key))
    message = PKCS7_pad(  # Padding function from Challenge 9
        b'It was the best of times, it was the worst of times, it was the age of wisdom, '
        b'it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, '
        b'it was the season of Light, it was the season of Darkness, it was the spring of hope, '
        b'it was the winter of despair.', len(key)
    )
    ciphertext = AES_CBC_encrypt(message, key, iv)
    plaintext = PKCS7_unpad(AES_CBC_decrypt(ciphertext, key, iv))  # Padding function from Challenge 9
    print(plaintext)
    
#     key = b'YELLOW SUBMARINE'
#     iv = bytes(16)
#     ciphertext = b''
#     with urlopen('https://cryptopals.com/static/challenge-data/10.txt') as data:
#         for line in data:
#             ciphertext += b64decode(line.rstrip())
    
#     plaintext = PKCS7_unpad(AES_CBC_decrypt(ciphertext, key, iv))  # Padding function from Challenge 9
#     print(plaintext)
    
main()

b'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair.'


### Challenge 11

An ECB/CBC detection oracle

In [11]:
import os
from random import randint

def encryption_oracle(plaintext):
    rand_key = os.urandom(16)
    front_pad = os.urandom(randint(5, 10))
    back_pad = os.urandom(randint(5, 10))
    data = PKCS7_pad(front_pad + plaintext + back_pad, 16)
    if randint(0, 1):
        return AES_ECB_encrypt(data, rand_key), 'ECB'
    else:
        return AES_CBC_encrypt(data, rand_key, iv=os.urandom(16)), 'CBC'

def detect_ECB_or_CBC(ciphertext, plaintext_length):
    score = detect_AES_ECB(ciphertext)  # from Challenge 8
    
    # If at least half of the blocks are duplicates, then we guess ECB mode.
    # The 50% requirement is somewhat arbitrary, but is based on Challenge 12.
    # For my code here, you can increase the requirement to 66% before it starts making mistakes
    if (score / (plaintext_length // 16)) * 100 >= 50:
        return 'ECB'
    else:
        return 'CBC'

def main():
    percentage = 0
    r = 10000
    for _ in range(r):
        message = b'A' * randint(100, 200)
        ciphertext, mode = encryption_oracle(message)
        potential_mode = detect_ECB_or_CBC(ciphertext, len(message))
        if mode == potential_mode:
            percentage += 1
    print(f"The correct mode was guessed {(percentage / r) * 100}% of the time.")

main()

The correct mode was guessed 100.0% of the time.


### Challenge 12

Byte-at-a-time ECB decryption (Simple)

In [64]:
import os
from base64 import b64decode

rand_key = os.urandom(16)
unknown_string = b64decode(
    b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg' +
    b'aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq' +
    b'dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg' +
    b'YnkK'
)

def encryption_oracle(plaintext):
    plaintext = PKCS7_pad(plaintext + unknown_string, len(rand_key))
    return AES_ECB_encrypt(plaintext, rand_key)

def get_block_and_pad_size():
    base_length = len(encryption_oracle(bytes()))
    pad_size = 1
    while True:
        data = b'A' * pad_size
        new_length = len(encryption_oracle(data))
        potential_block_size = new_length - base_length
        if potential_block_size:
            return potential_block_size, pad_size
        pad_size += 1

def match_next_byte(input_string, output, start, end):
    for c in range(256):
        guess_byte = chr(c).encode()
        guess_input = input_string + guess_byte
        guess_output = encryption_oracle(guess_input)[start:end]
        if guess_output == output:
            return guess_byte

def get_unknown_string(block_size, pad_size):
    base_length = len(encryption_oracle(bytes()))
    solved_string = b''
    for i in range(base_length-1, pad_size-1, -1):
        input_string = (b'A' * i)
        
        start = base_length - block_size
        end = base_length
        output = encryption_oracle(input_string)[start:end]
        
        solved_string += match_next_byte(input_string + solved_string, output, start, end)
    return solved_string

def main():
    block_size, pad_size = get_block_and_pad_size()
    assert detect_ECB_or_CBC(encryption_oracle(b'A' * 32), 32) == 'ECB'
    solved_string = get_unknown_string(block_size, pad_size)
    print(solved_string)

main()

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"
